##Setup and Functions

In [0]:
from pyspark.sql import Row,SparkSession
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import time
from pyspark.sql.functions import *
import numpy as np
import random
import datetime
import pandas as pd
import xgboost as xgb
import mlflow.xgboost
import math
import itertools

from sparkdl.xgboost import XgboostRegressor,XgboostClassifier
from sklearn.model_selection import RandomizedSearchCV

from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit, udf

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:162: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
 original_result = python_builtin_import(name, globals, locals, fromlist, level)
/databricks/python/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
 from pandas import MultiIndex, Int64Index

In [0]:
blob_container = "team06" # The name of your container created in https://portal.azure.com
storage_account = "apatel" # The name of your Storage account created in https://portal.azure.com
secret_scope = "team06" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "team06" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

In [0]:
# this is not used; it was for an alternate strategy we did not choose

def find_optimal_threshold_df(df, fold_num, search_center=0.6, search_bounds=0.2, granularity=5, times_to_zoom=4):
    """Finds optimal threshold for a model given a fold number and inserts new 
    prediction column based on this optimal threshold (scored using f1 score)"""
    
    prob_name = "probability_" + str(fold_num)
    pred_name = "prediction_" + str(fold_num)
    rev_pred_name = "rev_pred_" + str(fold_num)
    label_name = "label_" + str(fold_num)
    
    def ith_(v, i):
        try:
            return float(v[i])
        except ValueError:
            return None
    
    ith = udf(ith_, DoubleType())
    output = df.withColumn("del_prob",ith(prob_name, lit(1)))
    
    for i in range(times_to_zoom):
        search_space = np.linspace(search_center - search_bounds, search_center + search_bounds, granularity)
        best_score = 0
        prior_score = 0
        best_thresh = -1
        for threshold in search_space:
            test_df = output.select("_id",label_name,'del_prob')
            test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
            test_df = test_df.withColumnRenamed(label_name, "label")
            test_df = test_df.select('label','prediction')
            test_df.cache()
            test_metrics = MulticlassMetrics(test_df.rdd)
            f1_score = test_metrics.fMeasure(1.0,1.0)
            print("threshold:",threshold,"f1 score:",f1_score)
            if f1_score > best_score:
                best_score = f1_score
                best_thresh = threshold
            elif f1_score < prior_score:
                break
            prior_score = f1_score
        print("="*45)
        print("best score this level:", best_score, "at threshold", best_thresh)
        print("="*45)
        search_center = best_thresh
        search_bounds = search_bounds / 4
        test_df.unpersist()
        
    join_df = output.select("_id",label_name,'del_prob')
    join_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(best_thresh)), 1.0).otherwise(0.0))
    join_metric = join_df.withColumnRenamed(label_name, "label").select("label","prediction")
    join_metrics = MulticlassMetrics(join_metric.rdd)
    new_f1 = test_metrics.fMeasure(1.0,1.0)
    
    join_df = join_df.withColumnRenamed("prediction",rev_pred_name).select('_id',rev_pred_name)

    score_metrics = df.select(label_name, pred_name).withColumnRenamed(label_name, "label").withColumnRenamed(pred_name, "prediction")
    metrics = MulticlassMetrics(score_metrics.rdd)
    orig_f1 = metrics.fMeasure(1.0,1.0)
    print("="*45)
    print("For fold",fold_num)
    print("="*45)
    print("Original f1 score:",orig_f1)
    print("New f1 score:", new_f1)

    final_df = df.join(join_df,['_id'])

    return final_df

In [0]:
def find_optimal_threshold(df, search_center=0.6, search_bounds=0.2, granularity=5, times_to_zoom=4):
    """Finds optimal threshold for a model based on f1 score"""
    
    def ith_(v, i):
        try:
            return float(v[i])
        except ValueError:
            return None
    
    ith = udf(ith_, DoubleType())
    output = df.withColumn("del_prob",ith("probability", lit(1)))
    
    for i in range(times_to_zoom):
        search_space = np.linspace(search_center - search_bounds, search_center + search_bounds, granularity)
        best_score = 0
        prior_score = 0
        best_thresh = -1
        for threshold in search_space:
            test_df = output.select('label','del_prob')
            test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
            test_df = test_df.select('label','prediction')
            test_df.cache()
            test_metrics = MulticlassMetrics(test_df.rdd)
            f1_score = test_metrics.fMeasure(1.0,1.0)
            print("threshold:",threshold,"f1 score:",f1_score)
            if f1_score > best_score:
                best_score = f1_score
                best_thresh = threshold
            elif f1_score < prior_score:
                break
            prior_score = f1_score
        print("="*45)
        print("best score this level:", best_score, "at threshold", best_thresh)
        print("="*45)
        search_center = best_thresh
        search_bounds = search_bounds / 4
        test_df.unpersist()

    print("overall best threshold:", best_thresh, "with f1 score", best_score)
    return best_thresh

In [0]:
preds_1 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_9_fold_1').select("features","label","prediction","probability")
preds_2 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_9_fold_2').select("features","label","prediction","probability")
preds_3 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_9_fold_3').select("features","label","prediction","probability")
preds_4 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_9_fold_4').select("features","label","prediction","probability")
preds_5 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_9_fold_5').select("features","label","prediction","probability")

display(preds_1)

features,label,prediction,probability
"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 8, 9, 10, 11, 13, 15, 23, 24, 29, 31, 32, 33, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84), values -> List(604.0, 21.0, 979.0, 0.07, 229.0, 16.0, 0.11, 1.0, 1.0, 1.0, 1.0, 0.07, 0.11, 0.016721392451989205, 0.017058616124661325, 10272.0, 57.0, 10332.426267281106, 16000.0, 111.0, -17.0, 10187.728110599079, 10260.0, 1793.1946564885495, 0.064, 0.11, 5.75, 11.0, 10.0, 1.0, 82.0, 425.79, 564.19, 284.85, 5.75, 0.2045052531119649, 17.565871638131753, 1.0, 7.0, 3.0, 0.2306252035167698, 20.615678931943993, 1.0, 20.0, 25.0, 0.42857142857142855, 128.5142857142857, 1.0, 33.0, 1.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8526660203933716, 0.1473339945077896))"
"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 8, 9, 10, 11, 13, 15, 23, 29, 31, 32, 33, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84), values -> List(604.0, 21.0, 979.0, 0.14, 263.0, 19.0, 0.11, 1.0, 1.0, 1.0, 0.14, 0.11, 0.016721392451989205, 0.017058616124661325, 10278.0, 26.0, 10332.426267281106, 16000.0, 122.0, -28.0, 10187.728110599079, 10264.0, 1793.1946564885495, 0.167, 1.0, 11.0, 10.0, 1.0, 59.0, 301.3, 315.96, 202.32, 7.456603773584906, 0.2045052531119649, 17.565871638131753, 1.0, 7.0, 3.0, 0.2306252035167698, 20.615678931943993, 1.0, 20.0, 25.0, 0.42857142857142855, 128.5142857142857, 1.0, 33.0, 1.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8603250980377197, 0.13967491686344147))"
"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 7, 8, 9, 10, 11, 13, 15, 23, 27, 29, 31, 32, 33, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84), values -> List(604.0, 21.0, 979.0, 1.0, 0.06, 252.0, 18.0, 0.05, 1.0, 1.0, 1.0, 1.0, 0.06, 0.05, 0.016721392451989205, 0.017058616124661325, 10152.0, 41.0, 10332.426267281106, 16000.0, 161.0, 39.0, 10187.728110599079, 10139.0, 1793.1946564885495, 0.112, 1.0, 11.0, 10.0, 1.0, 165.0, 334.77, 341.53, 575.44, 2.921259842519685, 0.2045052531119649, 17.565871638131753, 1.0, 7.0, 3.0, 0.2306252035167698, 20.615678931943993, 1.0, 20.0, 25.0, 0.42857142857142855, 128.5142857142857, 1.0, 33.0, 1.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8675660490989685, 0.1324339658021927))"
"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 8, 9, 10, 11, 13, 15, 23, 29, 31, 32, 33, 35, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84), values -> List(604.0, 21.0, 979.0, 0.12, 273.0, 18.0, 0.11, 1.0, 1.0, 1.0, 0.12, 0.11, 0.016721392451989205, 0.017058616124661325, 3.0, 10112.0, 72.0, 10332.426267281106, 16000.0, 128.0, 94.0, 10187.728110599079, 10098.0, 1793.1946564885495, 0.08, 1.0, 11.0, 10.0, 1.0, 117.0, 313.68, 366.54, 212.88, 8.592592592592593, 0.2045052531119649, 17.565871638131753, 1.0, 7.0, 3.0, 0.2306252035167698, 20.615678931943993, 1.0, 20.0, 25.0, 0.42857142857142855, 128.5142857142857, 1.0, 33.0, 1.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8229661583900452, 0.17703382670879364))"
"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 8, 9, 10, 11, 13, 15, 23, 29, 31, 32, 33, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84), values -> List(604.0, 21.0, 979.0, 0.08, 278.0, 18.0, 0.16, 1.0, 1.0, 1.0, 0.08, 0.16, 0.016721392451989205, 0.017058616124661325, 10100.0, 51.0, 10332.426267281106, 16000.0, 133.0, 6.0, 10187.728110599079, 10088.0, 1793.1946564885495, 0.158, 12.0, 1.0, 11.0, 10.0, 1.0, 73.0, 275.96, 359.61, 236.22, 8.226618705035971, 0.2045052531119649, 

In [0]:
for i in range(1,6):
    preds_df = find_optimal_threshold(preds_df, search_center=0.6, search_bounds=0.2, granularity=5, times_to_zoom=4, fold_num=i)

In [0]:
preds_1 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_7_fold_1')
preds_1.cache()
preds_1_thresh = find_optimal_threshold(preds_1)
preds_1.unpersist()

preds_2 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_7_fold_2')
preds_2.cache()
preds_2_thresh = find_optimal_threshold(preds_2) 
preds_2.unpersist()

preds_3 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_7_fold_3')
preds_3.cache()
preds_3_thresh = find_optimal_threshold(preds_3) 
preds_3.unpersist()

preds_4 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_7_fold_4')
preds_4.cache()
preds_4_thresh = find_optimal_threshold(preds_4) 
preds_4.unpersist()

preds_5 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_7_fold_5')
preds_5.cache()
preds_5_thresh = find_optimal_threshold(preds_5) 
preds_5.unpersist()

threshold: 0.39999999999999997 f1 score: 0.5203970997120441
threshold: 0.5 f1 score: 0.5304115083480186
threshold: 0.6 f1 score: 0.5116969535265734
=============================================
best score this level: 0.5304115083480186 at threshold 0.5
=============================================
threshold: 0.45 f1 score: 0.529391322540363
threshold: 0.47500000000000003 f1 score: 0.5308142335581361
threshold: 0.5 f1 score: 0.5304115083480186
=============================================
best score this level: 0.5308142335581361 at threshold 0.47500000000000003
=============================================
threshold: 0.4625 f1 score: 0.5304061092763387
threshold: 0.46875 f1 score: 0.5307323490030981
threshold: 0.47500000000000003 f1 score: 0.5308142335581361
threshold: 0.48125000000000007 f1 score: 0.5308991916477752
threshold: 0.48750000000000004 f1 score: 0.5309079192842906
=============================================
best score this level: 0.5309079192842906 at threshold 0.48750000000000004
=============================================
threshold: 0.48437500000000006 f1 score: 0.5309065889764457
threshold: 0.4859375 f1 score: 0.5309232003242931
threshold: 0.48750000000000004 f1 score: 0.5309079192842906
=============================================
best score this level: 0.5309232003242931 at threshold 0.4859375
=============================================
overall best threshold: 0.4859375 with f1 score 0.5309232003242931
threshold: 0.39999999999999997 f1 score: 0.5249290177808605
threshold: 0.5 f1 score: 0.5454481734555878
threshold: 0.6 f1 score: 0.537349185122048
=============================================
best score this level: 0.5454481734555878 at threshold 0.5
=============================================
threshold: 0.45 f1 score: 0.5390605502445381
threshold: 0.47500000000000003 f1 score: 0.543288599056161
threshold: 0.5 f1 score: 0.5454481734555878
threshold: 0.525 f1 score: 0.5462426338578784
threshold: 0.55 f1 score: 0.5448925419276759
=============================================
best score this level: 0.5462426338578784 at threshold 0.525
=============================================
threshold: 0.5125000000000001 f1 score: 0.5461000499886569
threshold: 0.51875 f1 score: 0.5461739119209829
threshold: 0.525 f1 score: 0.5462426338578784
threshold: 0.53125 f1 score: 0.5460194872164287
=============================================
best score this level: 0.5462426338578784 at threshold 0.525
=============================================
threshold: 0.521875 f1 score: 0.5462399107045247
threshold: 0.5234375 f1 score: 0.5462185420751232
=============================================
best score this level: 0.5462399107045247 at threshold 0.521875
=============================================
overall best threshold: 0.521875 with f1 score 0.5462399107045247
threshold: 0.39999999999999997 f1 score: 0.5271912045253787
threshold: 0.5 f1 score: 0.5437246353307066
threshold: 0.6 f1 score: 0.5335682719174196
=============================================
best score this level: 0.5437246353307066 at threshold 0.5
=============================================
threshold: 0.45 f1 score: 0.539103190415869
threshold: 0.47500000000000003 f1 score: 0.5422148534433101
threshold: 0.5 f1 score: 0.5437246353307066
threshold: 0.525 f1 score: 0.5434167140646814
=============================================
best score this level: 0.5437246353307066 at threshold 0.5
=============================================
threshold: 0.4875 f1 score: 0.543151856858746
threshold: 0.49374999999999997 f1 score: 0.5434534135982256
threshold: 0.5 f1 score: 0.5437246353307066
threshold: 0.50625 f1 score: 0.5438249624319995
threshold: 0.5125 f1 score: 0.5437962334713086
=============================================
best score this level: 0.5438249624319995 at threshold 0.50625
=============================================
threshold: 0.5031249999999999 f1 score: 0.5437551699424343
threshold: 0.5046875 f1 score: 0.5437808405518225
threshold: 0.50625 f1 score: 0.5

##Re-Labeling on new thresholds

In [0]:
def infer_new_labels(df, threshold, name):
    """Based on input df and threshold, output df with re-inferred labels from new threshold."""
    def ith_(v, i):
        try:
            return float(v[i])
        except ValueError:
            return None
    
    ith = udf(ith_, DoubleType())
    output = df.withColumn("del_prob",ith("probability", lit(1)))
    
    metrics = MulticlassMetrics(df.select('label','prediction').rdd)
    orig_f1 = metrics.fMeasure(1.0,1.0)
    print("="*45)
    print("For",name)
    print("Original f1 score:",orig_f1)
    
    test_df = output.select('features','label','del_prob')
    test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
    test_df = test_df.select('features','label','prediction')
    test_df.cache()
    test_metrics = MulticlassMetrics(test_df.select('label','prediction').rdd)
    new_f1 = test_metrics.fMeasure(1.0,1.0)
    test_df.unpersist()
    
    print("New f1 score:", new_f1)
    
    return test_df

### Apply new labels and save out files

In [0]:
CV_name = 'fold_1'
save_name = 'xgboost_reinf_4_9_'+CV_name
preds_1 = spark.read.parquet(f'{blob_url}/xgboost_test_preds_4_9_fold_1')
preds_1.cache()
preds_1_reinf = infer_new_labels(df=preds_1, threshold=preds_1_thresh, name=CV_name)
preds_1.unpersist()
preds_1_reinf.cache()
preds_1_reinf.write.parquet(f"{blob_url}/{save_name}")
preds_1_reinf.unpersist()

CV_name = 'fold_2'
save_name = 'xgboost_reinf_4_9_'+CV_name
preds_2 = spark.read.parquet(f'{blob_url}/xgboost_test_preds_4_9_fold_2')
preds_2.cache()
preds_2_reinf = infer_new_labels(df=preds_2, threshold=preds_2_thresh, name=CV_name)
preds_2.unpersist()
preds_2_reinf.cache()
preds_2_reinf.write.parquet(f"{blob_url}/{save_name}")
preds_2_reinf.unpersist()

CV_name = 'fold_3'
save_name = 'xgboost_reinf_4_9_'+CV_name
preds_3 = spark.read.parquet(f'{blob_url}/xgboost_test_preds_4_9_fold_3')
preds_3.cache()
preds_3_reinf = infer_new_labels(df=preds_3, threshold=preds_3_thresh, name=CV_name)
preds_3.unpersist()
preds_3_reinf.cache()
preds_3_reinf.write.parquet(f"{blob_url}/{save_name}")
preds_3_reinf.unpersist()

CV_name = 'fold_4'
save_name = 'xgboost_reinf_4_9_'+CV_name
preds_4 = spark.read.parquet(f'{blob_url}/xgboost_test_preds_4_9_fold_4')
preds_4.cache()
preds_4_reinf = infer_new_labels(df=preds_4, threshold=preds_4_thresh, name=CV_name)
preds_4.unpersist()
preds_4_reinf.cache()
preds_4_reinf.write.parquet(f"{blob_url}/{save_name}")
preds_4_reinf.unpersist()

CV_name = 'fold_5'
save_name = 'xgboost_reinf_4_9_'+CV_name
preds_5 = spark.read.parquet(f'{blob_url}/xgboost_test_preds_4_9_fold_5')
preds_5.cache()
preds_5_reinf = infer_new_labels(df=preds_5, threshold=preds_5_thresh, name=CV_name)
preds_5.unpersist()
preds_5_reinf.cache()
preds_5_reinf.write.parquet(f"{blob_url}/{save_name}")
preds_5_reinf.unpersist()

=============================================
For fold_1
Original f1 score: 0.546036877165993
New f1 score: 0.5464268852356307
=============================================
For fold_2
Original f1 score: 0.5434717002619293
New f1 score: 0.5441613438913524
=============================================
For fold_3
Original f1 score: 0.5442935508483706
New f1 score: 0.5443423822462723
=============================================
For fold_4
Original f1 score: 0.5461277381257137
New f1 score: 0.5460808130253559
=============================================
For fold_5
Original f1 score: 0.5436193217409645
New f1 score: 0.5436609569200485
Out[30]: DataFrame[features: vector, label: double, prediction: double]

#Voting and Test Set Performance

In [0]:
# We realized after conducting inference that the order had become scrambled between the folds
# Not all was lost! After much testing, we determined that only 24 rows had duplicate features, and only 18 of those with duplicate labels
# Reasoning that a fold's prediction will always be consistent with the same features, we held fold 1 in place and joined the other folds to it based on features
# 

# read in files

preds_1_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_9_fold_1')
preds_2_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_9_fold_2')
preds_3_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_9_fold_3')
preds_4_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_9_fold_4')
preds_5_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_9_fold_5')

# verify lengths

print(preds_1_reinf.count())
print(preds_2_reinf.count())
print(preds_3_reinf.count())
print(preds_4_reinf.count())
print(preds_5_reinf.count())

7250472
7250472
7250472
7250472
7250472

In [0]:
all_preds = preds_1_reinf.withColumnRenamed("prediction","prediction_1").select("features","prediction_1","label")

all_preds = all_preds.alias("L").join(preds_2_reinf.alias("R").select("features","prediction","label")\
                                      .withColumnRenamed("prediction","prediction_2"), ['features', 'label'])

# drop duplicates with more than 2 matches

all_preds.createOrReplaceTempView("Preds")
all_preds = spark.sql("SELECT features, label, prediction_1, prediction_2 FROM (SELECT P.*, ROW_NUMBER() OVER(PARTITION BY features, label ORDER BY prediction_1 ASC) as rownum FROM Preds P) a WHERE a.rownum < 3")


all_preds = all_preds.alias("L").join(preds_3_reinf.alias("R").select("features","prediction","label")\
                                      .withColumnRenamed("prediction","prediction_3"), ['features', 'label'])

all_preds.createOrReplaceTempView("Preds")
all_preds = spark.sql("SELECT features, label, prediction_1, prediction_2, prediction_3 FROM (SELECT P.*, ROW_NUMBER() OVER(PARTITION BY features, label ORDER BY prediction_1 ASC) as rownum FROM Preds P) a WHERE a.rownum < 3")


all_preds = all_preds.alias("L").join(preds_4_reinf.alias("R").select("features","prediction","label")\
                                      .withColumnRenamed("prediction","prediction_4"), ['features', 'label'])

all_preds.createOrReplaceTempView("Preds")
all_preds = spark.sql("SELECT features, label, prediction_1, prediction_2, prediction_3, prediction_4 FROM (SELECT P.*, ROW_NUMBER() OVER(PARTITION BY features, label ORDER BY prediction_1 ASC) as rownum FROM Preds P) a WHERE a.rownum < 3")


all_preds = all_preds.alias("L").join(preds_5_reinf.alias("R").select("features","prediction","label")\
                                      .withColumnRenamed("prediction","prediction_5"), ['features', 'label'])

all_preds.createOrReplaceTempView("Preds")
all_preds = spark.sql("SELECT label, features, prediction_1, prediction_2, prediction_3, prediction_4, prediction_5 FROM (SELECT P.*, ROW_NUMBER() OVER(PARTITION BY features, label ORDER BY prediction_1 ASC) as rownum FROM Preds P) a WHERE a.rownum < 3")

In [0]:
# verify join results

all_preds.cache()

display(all_preds)
all_preds.count()

label,features,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5
0.0,"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 22, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 49, 50, 51, 52, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 78, 79, 80, 81, 83, 84), values -> List(1554.0, 20.0, 487.0, 1.0, 159.0, 1.0, 156.0, 1.0, 0.33, 401.0, 199.0, 0.21, 1.0, 1.0, 1.0, 1.0, 159.0, 1.0, 156.0, 0.33, 1.0, 0.21, 0.009816910506378085, 0.01025791928499623, 118.0, 10100.0, 82.0, 22000.0, 16093.0, -83.0, 10102.0, 10091.0, 1524.0, 0.111, 1.0, 26.0, 21.0, 1.0, 619.0, 253.07, 347.06, 251.83, 13.962666666666667, 0.20807924111123, 11.206206982632574, 3.0, 10.0, 0.19555506837302347, 13.98635279932038, 52.0, 125.0, 0.08695652173913043, 6.6521739130434785, 3091.0, 4412.0))",0.0,0.0,0.0,0.0,0.0
0.0,"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 78, 79, 81, 83, 84), values -> List(1212.0, 728.0, 607.0, 1.0, 24.0, 1.0, 17.0, 1.0, 0.17, 65.0, 684.0, 0.18, 1.0, 1.0, 1.0, 1.0, 1.0, 24.0, 1.0, 17.0, 0.17, 1.0, 0.18, 0.0034340306679597715, 0.004106753071779026, 10159.0, 51.0, 22000.0, 16000.0, -50.0, -100.0, 10170.035815268615, 9881.0, 936.1568998109641, 0.071, 1.0, 72.0, 61.0, 1.0, 488.0, 364.77, 244.85, 344.29, 16.705128205128204, 0.16533909524420967, 11.721165724304962, 8.0, 9.0, 0.13805686483199026, 10.893720541280219, 226.0, 230.0, 0.45454545454545453, 3391.0, 5258.0))",0.0,0.0,0.0,0.0,0.0
1.0,"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 22, 25, 26, 27, 28, 29, 30, 31, 32, 33, 49, 50, 51, 52, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 78, 79, 80, 81, 83, 84), values -> List(1999.0, 146.0, 1135.0, 1.0, 24.0, 1.0, 18.0, 1.0, 0.17, 241.0, 481.0, 0.14, 1.0, 1.0, 1.0, 1.0, 24.0, 1.0, 18.0, 0.17, 1.0, 0.14, 0.00930196273177531, 0.008631014346010992, 10173.0, 36.0, 7620.0, 16093.0, -122.0, 10173.0, 10116.0, 5791.0, 0.054, 1.5, 4.0, 28.0, 29.0, 1.0, 397.0, 329.88, 335.07, 373.94, 9.17117117117117, 0.20807924111123, 11.206206982632574, 3.0, 10.0, 0.18663221228016044, 11.848426411601357, 65.0, 188.0, 0.1391509433962264, 9.617924528301886, 2270.0, 3279.0))",0.0,0.0,0.0,0.0,0.0
0.0,"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 25, 26, 27, 28, 29, 30, 31, 32, 33, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 78, 79, 80, 81, 83, 84), values -> List(364.0, 36.0, 76.0, 1.0, 103.0, 1.0, 106.0, 1.0, 0.36, 324.0, 38.0, 0.31, 1.0, 1.0, 1.0, 103.0, 1.0, 106.0, 0.36, 1.0, 0.31, 0.008701554732167022, 0.009101861508585643, 10026.0, 36.0, 366.0, 12875.0, 256.0, 228.0, 10027.0, 10016.0, 244.0, 0.273, 1.333, 3.0, 31.0, 26.0, 1.0, 939.0, 228.69, 282.13, 181.05, 25.75811209439528, 0.16280382878038288, 12.261440632394063, 14.0, 11.0, 0.14269574059428314, 10.450193951719204, 230.0, 268.0, 0.12149532710280374, 5.841121495327103, 2734.0, 4785.0))",1.0,1.0,1.0,1.0,1.0
0.0,"Map(vectorType -> sparse, length -> 85, indices -> List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 25, 26, 27, 28, 29, 30, 31, 32, 33, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 78, 79, 80, 81, 83, 84), values -> List(587.0, 146.0, 599.0, 1.0, 21.0, 1.0, 23.0, 1.0, 0.24, 300.0, 220.0, 0.4, 1.0, 1.0, 1.0, 21.0, 1.0, 23.0, 0.24, 1.0, 0.4, 0.00948494656296053, 0.008076466597018312, 10148.0, 7620.0, 16093.0, 228.0, 206.0, 10149.0, 10093.0, 4267.0, 0.123, 0.2, 51.6, 5.0, 26.0, 29.0, 1.0, 437.0, 219.98, 302.39, 260.44, 62.15613382899628, 0.2398833534977601, 14.220697917069211, 5.0, 14.0, 0.22719161699381146, 14.414062796613386, 53.0, 213.0, 0.22287968441814596, 

Out[6]: 7250472

In [0]:
all_preds_f = all_preds.withColumn("weighted_pred", lit(0.15)*col('prediction_1') + lit(0.15)*col('prediction_2') + lit(0.2)*col('prediction_3') + lit(0.3)*col('prediction_4')+ lit(0.2)*col('prediction_5'))


all_preds_f = all_preds_f.withColumn("prediction", (col("weighted_pred") >= lit(0.5)).cast('double')).cache()

test_df = all_preds_f.select('prediction','label').cache()

In [0]:
all_preds_f.write.mode("overwrite").parquet(f"{blob_url}/xgboost_all_folds_weighted_preds_4_10")
# all_preds_f = spark.read.parquet(f'{blob_url}/xgboost_all_folds_weighted_preds_4_9')

In [0]:
test_df = preds_1_reinf.select('prediction','label')
test_metrics = MulticlassMetrics(test_df.rdd)
f1_score = test_metrics.fMeasure(1.0,1.0)

print("Fold 1 test set (2019) f1 score:", f1_score)

test_df = preds_2_reinf.select('prediction','label')
test_metrics = MulticlassMetrics(test_df.rdd)
f1_score = test_metrics.fMeasure(1.0,1.0)

print("Fold 2 test set (2019) f1 score:", f1_score)

test_df = preds_3_reinf.select('prediction','label')
test_metrics = MulticlassMetrics(test_df.rdd)
f1_score = test_metrics.fMeasure(1.0,1.0)

print("Fold 3 test set (2019) f1 score:", f1_score)

test_df = preds_4_reinf.select('prediction','label')
test_metrics = MulticlassMetrics(test_df.rdd)
f1_score = test_metrics.fMeasure(1.0,1.0)

print("Fold 4 test set (2019) f1 score:", f1_score)

test_df = preds_5_reinf.select('prediction','label')
test_metrics = MulticlassMetrics(test_df.rdd)
f1_score = test_metrics.fMeasure(1.0,1.0)

print("Fold 5 test set (2019) f1 score:", f1_score)


test_df = all_preds_f.select('prediction','label')

test_metrics = MulticlassMetrics(test_df.rdd)
f1_score = test_metrics.fMeasure(1.0,1.0)

print("All Folds weighted vote test set (2019) f1 score:", f1_score)

Fold 1 test set (2019) f1 score: 0.5464268852356307
Fold 2 test set (2019) f1 score: 0.5441613438913524
Fold 3 test set (2019) f1 score: 0.5443423822462723
Fold 4 test set (2019) f1 score: 0.5460808130253559
Fold 5 test set (2019) f1 score: 0.5436609569200485
All Folds weighted vote test set (2019) f1 score: 0.5473976873570806